In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import graphviz
from sklearn import ensemble , preprocessing , metrics , svm , tree , linear_model
from sklearn.neural_network import MLPClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import RMSprop ,Adam

Using TensorFlow backend.


## import train test data

In [4]:
data = pd.read_csv('C:\\Users\\USER\\github\\E.sun_conpetition\\data_set\\clean_data.csv')

In [5]:
data.head()

,Unnamed: 0,building_id,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,...,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
0,0,e3mMIMR3JJqCaXz1,8,21,18674,4,3,2,6271,2,...,58,1,157,1,2483,1,6011,1,34.469803,6.476038e+05
1,1,LgwzgklNvy4QCtq5,8,7,18800,5,1,2,7885,2,...,28,1,115,1,15872,1,32221,1,40.073573,3.321452e+06
2,2,ucIR2NLLsC3T650L,8,7,19289,4,1,2,6028,2,...,78,1,212,1,15760,1,32228,1,54.462081,9.570885e+06
3,3,jre1pJhcQj91Kdky,8,21,20385,24,0,2,18325,0,...,20,1,125,1,2568,1,7271,1,99.628966,1.421501e+07
4,4,rQpYpY9nRG7X5mmr,1,21,20657,2,4,2,6880,2,...,18,1,47,1,2587,1,7442,1,124.131236,7.627120e+05


In [6]:
data = data.iloc[:,1:]

### 將columns名稱儲存

In [7]:
columnslist = data.columns.tolist()

## 儲存 building_id 來做為辨識

In [8]:
buildid = data[columnslist[0]]

In [9]:
buildid[:5]

0    e3mMIMR3JJqCaXz1
1    LgwzgklNvy4QCtq5
2    ucIR2NLLsC3T650L
3    jre1pJhcQj91Kdky
4    rQpYpY9nRG7X5mmr
Name: building_id, dtype: object

## 將預測值 存取成train_y

In [10]:
train_y = data[columnslist[-1]]

In [11]:
train_y[:5]

0    6.476038e+05
1    3.321452e+06
2    9.570885e+06
3    1.421501e+07
4    7.627120e+05
Name: total_price, dtype: float64

## 將類別資料找出  並且以one_hot_encoding來處理

## 類別特徵分離出來 

In [12]:
columnslist

['building_id',
 'building_material',
 'city',
 'txn_dt',
 'total_floor',
 'building_type',
 'building_use',
 'building_complete_dt',
 'parking_way',
 'parking_area',
 'parking_price',
 'txn_floor',
 'land_area',
 'building_area',
 'town',
 'lat',
 'lon',
 'village_income_median',
 'town_population',
 'town_area',
 'town_population_density',
 'doc_Rate',
 'master_rate',
 'bachelor_rate',
 'jobschool_rate',
 'highschool_rate',
 'junior_rate',
 'elementary_rate',
 'born_rate',
 'death_date',
 'marriage_rate',
 'divorce_rate',
 'village',
 'N_50',
 'N_500',
 'N_1000',
 'N_5000',
 'N_10000',
 'I_10',
 'I_50',
 'I_index_50',
 'I_100',
 'I_250',
 'I_500',
 'I_index_500',
 'I_1000',
 'I_index_1000',
 'I_5000',
 'I_index_5000',
 'I_10000',
 'I_index_10000',
 'I_MIN',
 'II_10',
 'II_50',
 'II_index_50',
 'II_100',
 'II_250',
 'II_500',
 'II_index_500',
 'II_1000',
 'II_index_1000',
 'II_5000',
 'II_index_5000',
 'II_10000',
 'II_index_10000',
 'II_MIN',
 'III_10',
 'III_50',
 'III_index_50',
 '

In [13]:
list_dummy = ['building_material', 
              'city', 
              'building_type', 
              'building_use', 
              'parking_way', 
              'town', 
              'village']

In [14]:
def get_dummy(dataframe):
    dummy = pd.get_dummies(dataframe)
    return dummy

In [15]:
building_material = get_dummy(data['building_material']) 
city = get_dummy(data['city'])
building_type = get_dummy(data['building_type']) 
building_use = get_dummy(data['building_use']) 
parking_way = get_dummy(data['parking_way'])
town = get_dummy(data['town']) 
village = get_dummy(data['village'])

In [16]:
building_material.head()

,1,3,4,5,7,8,9,10,11
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0


In [17]:
list_dummy_var = [building_material,
                  city,
                  building_type,
                  building_use,
                  parking_way,
                  town,
                  village]


In [18]:
for item in list_dummy_var:
    print(len(item.columns))


9
11
5
10
3
211
2824


In [19]:
building_material_list = []
city_list = []
building_type_list = []
building_use_list = []
parking_way_list = []
town_list = []
village_list = []
for i in range(len(building_material.columns)):
    building_material_list.append('material_' + str(i))
for i in range(len(city.columns)):
    city_list.append('city_' + str(i))
for i in range(len(building_type.columns)):
    building_type_list.append('type_' + str(i))
for i in range(len(building_use.columns)):
    building_use_list.append('use_' + str(i))
for i in range(len(parking_way.columns)):
    parking_way_list.append('park_' + str(i))
for i in range(len(town.columns)):
    town_list.append('town_' + str(i))
for i in range(len(village.columns)):
    village_list.append('village' + str(i))

In [20]:
building_material.columns = building_material_list
city.columns = city_list
building_type.columns = building_type_list
building_use.columns = building_use_list
parking_way.columns = parking_way_list
town.columns = town_list
village.columns = village_list

print(building_material.columns)

Index(['material_0', 'material_1', 'material_2', 'material_3', 'material_4',
       'material_5', 'material_6', 'material_7', 'material_8'],
      dtype='object')


In [35]:
data_merge = pd.concat([data,building_material,city,building_type,building_use,parking_way,town,village],axis=1)
data_merge_no_vill = pd.concat([data,building_material,city,building_type,building_use,parking_way,town],axis=1)

## drop the category columns 

In [36]:
data_merge = data_merge.drop(columns = list_dummy)
data_merge_no_vill = data_merge_no_vill.drop(columns = list_dummy)

# Check the dataframe

In [37]:
data_merge.head()

,building_id,txn_dt,total_floor,building_complete_dt,parking_area,parking_price,txn_floor,land_area,building_area,lat,...,village2814,village2815,village2816,village2817,village2818,village2819,village2820,village2821,village2822,village2823
0,e3mMIMR3JJqCaXz1,18674,4,6271,0.0,0.000000,3.0,18.144460,3.418175,-39.14,...,0,0,0,0,0,0,0,0,0,0
1,LgwzgklNvy4QCtq5,18800,5,7885,0.0,0.000000,5.0,11.387227,4.041309,-37.66,...,0,0,0,0,0,0,0,0,0,0
2,ucIR2NLLsC3T650L,19289,4,6028,0.0,0.000000,1.0,21.426802,5.584279,-37.67,...,0,0,0,0,0,0,0,0,0,0
3,jre1pJhcQj91Kdky,20385,24,18325,0.0,81138.889762,13.0,11.387227,13.563031,-39.13,...,0,0,0,0,0,0,0,0,0,0
4,rQpYpY9nRG7X5mmr,20657,2,6880,0.0,0.000000,0.5,61.306524,4.688108,-39.24,...,0,0,0,0,0,0,0,0,0,0


In [38]:
data_merge_no_vill.head()

,building_id,txn_dt,total_floor,building_complete_dt,parking_area,parking_price,txn_floor,land_area,building_area,lat,...,town_201,town_202,town_203,town_204,town_205,town_206,town_207,town_208,town_209,town_210
0,e3mMIMR3JJqCaXz1,18674,4,6271,0.0,0.000000,3.0,18.144460,3.418175,-39.14,...,0,1,0,0,0,0,0,0,0,0
1,LgwzgklNvy4QCtq5,18800,5,7885,0.0,0.000000,5.0,11.387227,4.041309,-37.66,...,0,0,0,0,0,0,0,0,0,0
2,ucIR2NLLsC3T650L,19289,4,6028,0.0,0.000000,1.0,21.426802,5.584279,-37.67,...,0,0,0,0,0,0,0,0,0,0
3,jre1pJhcQj91Kdky,20385,24,18325,0.0,81138.889762,13.0,11.387227,13.563031,-39.13,...,0,0,0,0,0,0,1,0,0,0
4,rQpYpY9nRG7X5mmr,20657,2,6880,0.0,0.000000,0.5,61.306524,4.688108,-39.24,...,0,0,0,0,0,0,0,0,0,0


# save the data into csv

In [39]:
data_merge.to_csv('dummy_data.csv')
data_merge_no_vill.to_csv('dummy_data_novill.csv')

In [40]:
train_y.to_csv('train_y.csv')

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
